### Overview
In this notebook, we will look at the performance of BoolQ, PIQA and OpenBookQA

In [1]:
# Load vLLM for infernce
from datasets import load_from_disk, Dataset, load_dataset
import random
import pandas as pd
import argparse
import warnings
warnings.filterwarnings("ignore")
from vllm import LLM, SamplingParams
import torch
seed=42

In [2]:
model_name = "/project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/self_rewarding/LLM_Alignment/safety_llama_paper/models/safety_llama_replicate_500_safe/merged_model"
model = LLM(model=model_name,
                    tokenizer=model_name, 
                    tensor_parallel_size=torch.cuda.device_count(), 
                    seed=seed,
                    gpu_memory_utilization=0.9, 
                    dtype=torch.float16,
                    enforce_eager=True,
                    max_model_len=1024 # 512 is small for the BoolQ dataset, so changing it to 1024
        )

OSError: Incorrect path_or_model_id: '/project/pi_hongyu_umass_edu/zonghai/clinical-llm-alignment/durga_sandeep/self_rewarding/LLM_Alignment/safety_llama_paper/models/safety_llama_replicate_500_safe/merged_model'. Please provide either the path to a local folder or the repo_id of a model on the Hub.

In [ ]:

# helper functions
def get_prompt(example, data = 'boolq'):
    if data == 'boolq':
        prompt = f"### Instruction: Answer the following question (True/False) based on the passage.\n\n### Passage:\n{example['passage']}\n\n### Question: {example['question']}\n\n### Answer: "
    elif data == 'obqa':
        prompt = f"### Instruction: Answer the following mulitple-choice question (A/B/C/D).\n\n### Question:\n{example['question_stem']}\n\n### Choices: \n{example['choices']}\n### Answer: "
    elif data == 'piqa':
        prompt = f"### Instruction: Answer the following mulitple-choice question (A/B).\n\n### Question:\n{example['goal']}\n\n### Choices: \n{example['choices']}\n\n### Answer: "
    else:
        print("############# Issue with the prompt")
        return ""
        
    return prompt


def get_obqa_label(example):
    res = ""
    for i in range(len(example['text'])):
        res += f"{example['label'][i]}) {example['text'][i]}\n"
    return res

def preprocess_response_true_or_false(text):
    text = text.lower()
    if 'true' in text:
        return True
    if 'false' in text:
        return False
    return 'Other'

def preprocess_response_choices(text):
    text = text.lower()
    
    if ('a' in text) and ('b' not in text) and ('c' not in text) and ('d' not in text):
        return 'A'
    if ('a' not in text) and ('b' in text) and ('c' not in text) and ('d' not in text):
        return 'B'
    if ('a' not in text) and ('b' not in text) and ('c' in text) and ('d' not in text):
        return 'C'
    if ('a' not in text) and ('b' not in text) and ('c' not in text) and ('d' in text):
        return 'D'
    return 'Others'

def preprocess_piqa_response(text):
    text = text.lower()
    
    if ('0' in text) and ('1' not in text):
        return 0
    if ('1' in text) and ('0' not in text):
        return 1
    return 'Other'

### BoolQ

In [ ]:
# BoolQ dataset
bool_q = load_dataset("google/boolq")
boolq = pd.concat([bool_q['train'].to_pandas(), bool_q['validation'].to_pandas()])
print(boolq.shape)
boolq.head(2)

(12697, 3)


,question,answer,passage
0,do iran and afghanistan speak the same language,True,"Persian (/ˈpɜːrʒən, -ʃən/), also known by its ..."
1,do good samaritan laws protect those who help ...,True,Good Samaritan laws offer legal protection to ...


In [ ]:
# Add the respone prompt
boolq['inference_prompt'] = boolq.apply(lambda x: get_prompt(x, 'boolq'), 1)

In [ ]:
print(boolq['inference_prompt'].iloc[2])

### Instruction: Answer the following question (True/False) based on the passage.

### Passage:
Windows Movie Maker (formerly known as Windows Live Movie Maker in Windows 7) is a discontinued video editing software by Microsoft. It is a part of Windows Essentials software suite and offers the ability to create and edit videos as well as to publish them on OneDrive, Facebook, Vimeo, YouTube, and Flickr.

### Question: is windows movie maker part of windows essentials

### Answer: 


In [ ]:
boolq_sampling = SamplingParams(n=1, 
                                 max_tokens=3,
                                 # top_k=40,
                                 top_p=0.9,
                                 temperature=0.0,
                                 # frequency_penalty=1.0
                                )

In [ ]:
output = model.generate(boolq['inference_prompt'].tolist()[0], boolq_sampling)
torch.cuda.empty_cache()

Processed prompts: 100%|██████████| 1/1 [00:00<00:00,  9.54it/s]


In [ ]:
output[0].outputs[0].text

'\nFalse.'

In [ ]:
output = model.generate(boolq['inference_prompt'].tolist(), boolq_sampling)
torch.cuda.empty_cache()

Processed prompts:   2%|▏         | 193/12697 [00:10<11:29, 18.15it/s]

WARNING 05-03 20:12:53 scheduler.py:195] Input prompt (1072 tokens) is too long and exceeds limit of 1024


Processed prompts:  99%|█████████▉| 12561/12697 [11:55<00:07, 17.85it/s]

WARNING 05-03 20:24:38 scheduler.py:195] Input prompt (1402 tokens) is too long and exceeds limit of 1024


Processed prompts: 100%|██████████| 12697/12697 [12:02<00:00, 17.59it/s]


In [ ]:
boolq_results = []
for ele in output:
    boolq_results.append(ele.outputs[0].text)
boolq['generated_response'] = boolq_results
boolq['predicted_label'] = boolq['generated_response'].apply(preprocess_response_true_or_false)
boolq['predicted_label'] = boolq['predicted_label'].apply(str)

In [ ]:
boolq['predicted_label'].value_counts()

predicted_label
True     8520
False    3584
Other     593
Name: count, dtype: int64

In [ ]:
boolq['answer'].value_counts(normalize=True)

answer
True     0.622746
False    0.377254
Name: proportion, dtype: float64

In [ ]:
boolq[boolq['answer'].apply(str) == boolq['predicted_label']].shape[0]*100.0/boolq.shape[0]

71.08765850200835

In [ ]:
boolq.to_csv("./results/boolq_500_safety.csv", index=False)

### OpenBookQA

In [ ]:
# OpenBookQA
ob_qa = load_dataset("allenai/openbookqa")
obqa = pd.concat([ob_qa['train'].to_pandas(), ob_qa['validation'].to_pandas()])
print(obqa.shape)
obqa.head(2)

(5457, 4)


,id,question_stem,choices,answerKey
0,7-980,The sun is responsible for,"{'text': ['puppies learning new tricks', 'chil...",D
1,7-584,When standing miles away from Mount Rushmore,"{'text': ['the mountains seem very close', 'th...",D


In [ ]:
obqa['choices'] = obqa['choices'].apply(get_obqa_label)
obqa['inference_prompt'] = obqa.apply(lambda x: get_prompt(x, 'obqa'), 1)

In [ ]:
obqa_sampling = SamplingParams(n=1, 
                               max_tokens=2,
                               top_p=0.9,
                               temperature=0.0,
                               # frequency_penalty=1.0
                               )

In [ ]:
import random
for i in range(5):
    index = random.randint(0, len(obqa)-1)
    output = model.generate(obqa['inference_prompt'].tolist()[index], obqa_sampling)
    torch.cuda.empty_cache()
    print(output[0].outputs[0].text, obqa['answerKey'].tolist()[index])

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.09it/s]



C C


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.05it/s]



B A


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 15.61it/s]


...


C A


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.95it/s]



C C


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.84it/s]


C A


In [ ]:
output = model.generate(obqa['inference_prompt'].tolist(), obqa_sampling)
torch.cuda.empty_cache()

Processed prompts: 100%|██████████| 5457/5457 [02:05<00:00, 43.66it/s]


In [ ]:
all_results = []
for ele in output:
    all_results.append(ele.outputs[0].text)
obqa['generated_response'] = all_results
obqa['predicted_label'] = obqa['generated_response'].apply(lambda x: x.replace("\n", ""))
obqa['predicted_label'] = obqa['predicted_label'].apply(preprocess_response_choices)
obqa.predicted_label.value_counts()

predicted_label
C         4654
B          479
D          256
A           65
Others       3
Name: count, dtype: int64

In [ ]:
obqa['answerKey'].value_counts(normalize=True)

answerKey
A    0.275609
B    0.247389
D    0.246839
C    0.230163
Name: proportion, dtype: float64

In [ ]:
obqa[obqa['answerKey'] == obqa['predicted_label']].shape[0]*100.0/obqa.shape[0]

30.071467839472238

In [ ]:
obqa.to_csv("./results/obqa_500_safety.csv", index=False)

### PIQA

In [ ]:
# PIQA dataset
piqa_data = load_dataset("piqa")
piqa = pd.concat([piqa_data['train'].to_pandas(), piqa_data['validation'].to_pandas(), piqa_data['test'].to_pandas()])
piqa = piqa[piqa['label'] != -1]
print(piqa.shape)
piqa.head(2)

(17951, 4)


,goal,sol1,sol2,label
0,"When boiling butter, when it's ready, you can",Pour it onto a plate,Pour it into a jar,1
1,"To permanently attach metal legs to a chair, y...",Weld the metal together to get it to stay firm...,Nail the metal together to get it to stay firm...,0


In [ ]:
piqa.label.value_counts()

label
1    8988
0    8963
Name: count, dtype: int64

In [ ]:
piqa['choices'] = piqa.apply(lambda x: f"0) {x['sol1']}\n1) {x['sol2']}", 1)
piqa['inference_prompt'] = piqa.apply(lambda x: get_prompt(x, 'piqa'), 1)

In [ ]:
piqa_sampling = SamplingParams(n=1, 
                               max_tokens=2,
                               top_p=0.9,
                               temperature=0.0,
                               # frequency_penalty=1.0
                               )

In [ ]:
import random
for i in range(5):
    index = random.randint(0, len(piqa)-1)
    output = model.generate(piqa['inference_prompt'].tolist()[index], piqa_sampling)
    torch.cuda.empty_cache()
    print(output[0].outputs[0].text, piqa['label'].tolist()[index])

Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.41it/s]



1 1


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.66it/s]



0 1


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 12.63it/s]



0 0


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 15.75it/s]



1 0


Processed prompts: 100%|██████████| 1/1 [00:00<00:00, 13.62it/s]


1 1


In [ ]:
output = model.generate(piqa['inference_prompt'].tolist(), piqa_sampling)
torch.cuda.empty_cache()

Processed prompts:  31%|███       | 5505/17951 [03:08<06:49, 30.41it/s]

WARNING 05-03 20:37:56 scheduler.py:195] Input prompt (1341 tokens) is too long and exceeds limit of 1024


Processed prompts:  74%|███████▎  | 13208/17951 [07:28<02:14, 35.22it/s]

WARNING 05-03 20:42:15 scheduler.py:195] Input prompt (1322 tokens) is too long and exceeds limit of 1024


Processed prompts: 100%|██████████| 17951/17951 [10:11<00:00, 29.35it/s]


In [ ]:
all_results = []
for ele in output:
    all_results.append(ele.outputs[0].text)
piqa['generated_response'] = all_results
piqa['predicted_label'] = piqa['generated_response'].apply(preprocess_piqa_response)
piqa['predicted_label'] = piqa['predicted_label'].apply(str)
piqa.predicted_label.value_counts()

predicted_label
1        10422
0         7497
Other       32
Name: count, dtype: int64

In [ ]:
piqa['label'].value_counts(normalize=True)

label
1    0.500696
0    0.499304
Name: proportion, dtype: float64

In [ ]:
piqa[piqa['label'].apply(str) == piqa['predicted_label']].shape[0]*100.0/piqa.shape[0]

61.294635396356746

In [ ]:
piqa.to_csv("./results/piqa_500_safety.csv", index=False)

### First-Token Probability ?